# Algoritmos de optimización - Seminario<br>
<b>Nombre y Apellidos: </b> Yeray Quiles Ferrández <br>
> 1. Organizar sesiones de doblaje <br>

GitHub: https://github.com/YerayQuiles/optimizacion_algoritmos <br>



## Enunciado


<b>Descripción del problema: </b><br>
Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible. Los datos son:
- <b>Número de actores: </b> `10`
- <b> Número de tomas: </b> `30`
- <b>Actores/Tomas:</b> https://docs.google.com/spreadsheets/d/1Ipn6IrbQP4ax8zOnivdBIw2lN0JISkJG4fXndYd27U0/edit?gid=0#gid=0
         

In [96]:
from itertools import combinations
from math import inf

import pandas as pd
import math
import random
import time
from copy import deepcopy

#### Obtener los datos de la tabla

In [97]:
sheet_id = "1Ipn6IrbQP4ax8zOnivdBIw2lN0JISkJG4fXndYd27U0"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv"

df = pd.read_csv(url)
df_numeric = df[pd.to_numeric(df.iloc[:, 0], errors='coerce').notna()]

tomas_actores = {}
for _, row in df_numeric.iterrows():
    toma = int(row.iloc[0])
    actores = {i for i in range(1, 11) if row.iloc[i] == 1}
    tomas_actores[toma] = actores

En la tabla: 
- `1` el actor participa en la toma
- `0` el actor NO participa en la toma     


| Toma | Actor 1 | Actor 2 | Actor 3 | Actor 4 | Actor 5 | Actor 6 | Actor 7 | Actor 8 | Actor 9 | Actor 10 | Total |
|------|---------|---------|---------|---------|---------|---------|---------|---------|---------|----------|-------|
| 1 | 1 | 1 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 5 |
| 2 | 0 | 0 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 3 |
| 3 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 3 |
| 4 | 1 | 1 | 0 | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 4 |
| 5 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 3 |
| 6 | 1 | 1 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 4 |
| 7 | 1 | 1 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 4 |
| 8 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 3 |
| 9 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 3 |
| 10 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 4 |
| 11 | 1 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 5 |
| 12 | 1 | 1 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 5 |
| 13 | 1 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 3 |
| 14 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 3 |
| 15 | 1 | 1 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 3 |
| 16 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1 | 2 |
| 17 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 2 |
| 18 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 2 |
| 19 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 2 |
| 20 | 1 | 0 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 4 |
| 21 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 2 |
| 22 | 1 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 4 |
| 23 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 2 |
| 24 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 2 |
| 25 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1 | 4 |
| 26 | 1 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 4 |
| 27 | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 2 |
| 28 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 2 |
| 29 | 1 | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 3 |
| 30 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 2 |
|------|---------|---------|---------|---------|---------|---------|---------|---------|---------|----------|-------|
| **TOTAL** | **22** | **14** | **13** | **15** | **11** | **8** | **3** | **4** | **2** | **2** | **94** |


## ¿Cuántas posibilidades hay sin tener en cuenta las restricciones? 



El objetivo sería organizar el doblaje de una película con un total de 30 tomas y 10 actores. Para calcular el número de posibilidades sin restricciones, es necesario estudiar el orden de las tomas a grabar y la distribución de las sesiones de grabación a lo largo de los días.

---

### Opción 1: n! (Permutaciones)

Si únicamente consideramos el orden de las tomas:
- Tendríamos n! = 30! permutaciones posibles.
- Se asume que el orden de grabación es lo único relevante para la planificación de las sesiones.
- No se considera cómo se distribuyen las tomas en múltiples días.
- Sería válido si todas las tomas se graban en un solo día.

**Resultado:** 265,252,859,812,191,058,636,308,480,000,000 posibilidades

Esta opción no es suficiente para modelar el problema, ya que la distribución de los díases es un factor esencial para la planificación.



In [98]:
n = 30  # Parámetros del problema

# n! 
solo_factorial = math.factorial(n)

print(f'Opción 1: n! = {solo_factorial:,}')

Opción 1: n! = 265,252,859,812,191,058,636,308,480,000,000



### Opción 2: n! × 2^(n-1) (Secuenciación + Distribución)

Si asumimos que el número total de ordenaciones posibles es relevante, entonces se trata de un problema de permutaciones sin repetición.

#### Distribución de las sesiones de grabación
- En cada sesión de grabación de un día, se pueden grabar un número distinto de tomas.
- Las grabaciones son únicas, es decir, no pueden repetirse. Por tanto, una vez grabada una toma se eliminará del conjunto de tomas a grabar en las siguientes sesiones.
- Sea d∈{0,…,n−1} el número de particiones en que se pueden distribuir las sesiones de grabación de las tomas. Por ejemplo:
  - Si se graban todas las tomas en un solo día: **d=0** (sin particiones), si las tomas se graban en dos sesiones: **d=1** (una partición), etc
- El número de posibles ordenaciones de la partición d=0 sería C(n-1,0), el de la partición d=1 sería C(n-1,1), y así hasta el último número combinatorio.
- Por tanto, el número total de posibles particiones será la suma de todas las posibles distribuciones: **∑(j=0 hasta n-1) C(n-1,j)**

#### Número total de posibilidades sin restricción

El número total de posibles ordenaciones de las tomas a lo largo de los días se obtiene calculando el producto del orden de las tomas por la distribución de las sesiones de grabación, que por identidad binomial es:

**Núm. posibilidades = n! × ∑(j=0 hasta n-1) C(n-1,j) = n! × 2^(n-1)**


In [99]:
n = 30  # Parámetros del problema

# n! × 2^(n-1) 
orden_tomas = math.factorial(n)
distribucion_dias = 2**(n-1)
total_posibilidades = orden_tomas * distribucion_dias

# Resultados
print(f'Opción 2: n! × 2^(n-1) = {total_posibilidades:,}')
print(f'Factor de diferencia: {total_posibilidades/solo_factorial:,}')

Opción 2: n! × 2^(n-1) = 142,406,544,757,979,162,368,320,409,970,933,760,000,000
Factor de diferencia: 536,870,912.0



### Comparación de Enfoques

| Aspecto | n! | n! × 2^(n-1) |
|---------|---------------|-------------------------|
| **Considera orden de tomas** | Sí | Sí |
| **Considera distribución en días** | No | Sí |
| **Realismo para doblaje** | Limitado | Completo |
| **Resultado para n=30** | ~2.65 × 10^32 | ~1.42 × 10^41 |

### Resultados

| Opción | Fórmula | Resultado |
|--------|---------|-----------|
| **Opción 1 (descartada)** | n! | 265,252,859,812,191,058,636,308,480,000,000 |
| **Opción 2 (correcta)** | n! × 2^(n-1) | 142,406,544,757,979,162,368,320,409,970,933,760,000,000 |

La opción correcta considera tanto el orden de grabación como la distribución temporal, siendo 536,870,912 veces mayor que la opción que solo considera el orden.

**Respuesta final:** 142,406,544,757,979,162,368,320,409,970,933,760,000,000 posibilidades

## ¿Cuántas posibilidades hay teniendo en cuenta todas las restricciones?



El objetivo sería organizar el doblaje de una película con un total de 30 tomas y 10 actores, con un máximo de 6 tomal por día y el objetivo de minimizar el coste total. Para calcular el número de posibilidades con restricciones, es necesario estudiar el orden de las tomas a grabar y la distribución de las sesiones de grabación a lo largo de los días.

---

### Opción 1: Combinaciones con orden de días relevante

Si consideramos que el orden cronológico de los días es importante:

- No hay restricciones sobre cómo deben ser grabadas las tomas dentro de la sesión, se asume que el orden de grabación de las tomas dentro de cada día es indiferente.
- El orden de los días sí se considera relevante.

#### Distribución de los días de grabación
- El mínimo número de días necesarios es **r = ⌈n/k⌉ = ⌈30/6⌉ = 5** días.
- Para optimizar costes, asumimos sesiones de **k = 6** tomas cada día.
- Las grabaciones son únicas,una vez grabada se elimina del conjunto de tomas disponibles.
- Para el día **j+1**, tenemos **C(n-k·j, k)** posibilidades, donde **j ∈ {0,1,2,3,4}**.

#### Número total de posibilidades (orden relevante)

**Núm. posibilidades = ∏[j=0 hasta 4] C(30-6·j, 6)**

**Resultado:** 1,370,874,167,589,326,400 posibilidades


### Opción 2: Combinaciones con orden de días irrelevante

- El orden de grabación de las tomas dentro de cada día es indiferente.
- El orden de los días tampoco es relevante.

#### Distribución de las sesiones de grabación
- Mantenemos la configuración de 5 días con 6 tomas cada uno.
- Las configuraciones que solo difieren en el orden de los días se consideran equivalentes.

#### Corrección por equivalencia de días

Dado que el orden de los días no importa, debemos dividir el resultado de la Opción 1 por el número de permutaciones de los 5 días:

**Núm. posibilidades = [∏[j=0 hasta 4] C(30-6·j, 6)] ÷ 5!**


In [100]:
n = 30
k = 6
num_dias = n // k  # = 5 días

# Cálculo del producto de combinaciones
num_posibilidades_con_orden = 1
for j in range(0, num_dias):
    combinacion = math.comb(n - k*j, k)
    num_posibilidades_con_orden *= combinacion
    print(f"C({n - k*j}, {k}) = {combinacion:,}")

# Corrección por orden irrelevante de días
factorial_dias = math.factorial(num_dias)
num_posibilidades_sin_orden = num_posibilidades_con_orden // factorial_dias

print(f"Con orden: {num_posibilidades_con_orden:,}")
print(f"Factorial 5!: {factorial_dias}")
print(f"Sin orden: {num_posibilidades_sin_orden:,}")


C(30, 6) = 593,775
C(24, 6) = 134,596
C(18, 6) = 18,564
C(12, 6) = 924
C(6, 6) = 1
Con orden: 1,370,874,167,589,326,400
Factorial 5!: 120
Sin orden: 11,423,951,396,577,720



### Comparación de Enfoques

| Aspecto | Opción 1 (Orden relevante) | Opción 2 (Orden irrelevante) |
|---------|----------------------------|-------------------------------|
| **Considera restricción k≤6** | Sí | Sí |
| **Considera objetivo de costes** | Parcialmente | Completamente |
| **Distingue días equivalentes** | Sí  | No  |
| **Realismo para doblaje** | Limitado | Completo |
| **Resultado para n=30, k=6** | ~1.37 × 10¹⁵ | ~1.14 × 10¹³ |


### Resultados

| Opción | Fórmula | Resultado |
|--------|---------|-----------|
| **Opción 1** | ∏[j=0 hasta 4] C(30-6·j, 6) | 1,370,874,167,589,326,400 |
| **Opción 2** | [∏[j=0 hasta 4] C(30-6·j, 6)] ÷ 5! | 11,423,951,396,577,720 |

La opción correcta evita el sobreconteo de configuraciones equivalentes, siendo 120 (5!) veces menor que la opción que distingue incorrectamente el orden de días.


**Respuesta final:** 11,423,951,396,577,720 posibilidades

## ¿Cual es la estructura de datos que mejor se adapta al problema?



### Primera aproximación para definir la estructura de los datos:

Inicialmente se pensó en trabajar con la matriz de datos como un **array bidimensional de numpy** o un **dataframe de pandas** de manera que:
* en las filas tuviese como índice el número de toma (1 a 30),
* en las columnas tuviese como nombre el número de actor (1 a 10) y  

La solución se presentaría como una lista de sublistas donde cada sublista estaría formada por un número de tomas (≤6 tomas) para una sesión específica y se presentaría tantas sublistas como sesiones de grabación se considerase. Por ejemplo:

`[[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12], [13, 14, 15, 16, 17, 18], [19, 20, 21, 22, 23, 24], [25, 26, 27, 28, 29, 30]]`

No obstante, al experimentar las primeras dificultades en los diferentes bucles de optimización y analizar el rendimiento de la primera aproximación propuesta, se tomó la decisión de considerar una estructura alternativa.

### Estructura final de los datos:

En base a la matriz descrita en el apartado anterior se llevó a cabo una transformación para montar una **estructura híbrida de diccionarios con conjuntos** donde:
* cada clave es un número de toma (1 a 30) y
* cada valor del diccionario es un **conjunto (set)** con los números de actores que participan en esa toma.

Por ejemplo:
```python
{
    1: {1, 2, 3, 4, 5},     # Toma 1: actores 1,2,3,4,5
    2: {3, 4, 5},           # Toma 2: actores 3,4,5  
    3: {2, 5, 7},           # Toma 3: actores 2,5,7
    # ... hasta toma 30
}
```

Esta estructura elimina completamente el almacenamiento de ceros y aprovecha las operaciones nativas de conjuntos de Python para uniones eficientes.

La estructura final representa un equilibrio óptimo entre eficiencia computacional, simplicidad  y escalabilidad para el problema específico de coordinación de doblaje planteado, donde solo se almacenan las 94 participaciones reales vs 300 posiciones de la matriz.

In [101]:
tomas_actores

{1: {1, 2, 3, 4, 5},
 2: {3, 4, 5},
 3: {2, 5, 7},
 4: {1, 2, 7, 8},
 5: {2, 4, 8},
 6: {1, 2, 4, 5},
 7: {1, 2, 4, 5},
 8: {1, 2, 6},
 9: {1, 2, 4},
 10: {1, 2, 6, 9},
 11: {1, 2, 3, 5, 8},
 12: {1, 2, 3, 4, 6},
 13: {1, 4, 5},
 14: {1, 3, 6},
 15: {1, 2, 7},
 16: {4, 10},
 17: {1, 3},
 18: {3, 6},
 19: {1, 3},
 20: {1, 3, 4, 5},
 21: {6, 8},
 22: {1, 2, 3, 4},
 23: {1, 3},
 24: {3, 6},
 25: {1, 2, 4, 10},
 26: {1, 3, 5, 9},
 27: {4, 5},
 28: {1, 4},
 29: {1, 5, 6},
 30: {1, 4}}

## ¿Cuál es la función objetivo?



La función objetivo representa el nucleo del problema de optimización de coordinación de doblaje. Su diseño debe capturar la realidad económica del proceso: minimizar el coste total de producción expresado en días-actor, donde cada unidad representa el coste de un actor trabajando durante una sesión completa de grabación.

---

Los actores cobran una tarifa fija por día independientemente de cuántas tomas graben. Por lo que si un actor participa en una sola toma del día, cobrará el día completo igual que si participara en las seis tomas máximas permitidas. Esto determina que la función objetivo debe enfocarse en minimizar el número total de actores únicos que trabajan cada día, no el número de participaciones individuales.

La función objetivo que se implementó, llamada `coste`, determina el coste total asociado a una solución propuesta para la programación de las sesiones de doblaje. La función calcula cuántos actores únicos participan en cada sesión y suma los costes de todas las sesiones para obtener el coste total de la configuración.

### Cálculo de la función

**1. Iteración por sesiones**: Para cada sesión de grabación en la configuración, se calcula independientemente su coste asociado.

**2. Unión de actores por sesión**: Dentro de cada sesión, se identifican todos los actores únicos necesarios mediante la unión de los conjuntos de actores de cada toma. 

**3. Cálculo de coste marginal**: El coste de cada sesión se obtiene multiplicando el número de actores únicos por el salario diario. En el caso base, se asume salario_diario=1 para obtener directamente el número de días-actor.

**4. Agregación final**: El coste total se calcula sumando los costes marginales de todas las sesiones.


In [102]:
def coste(configuracion, tomas_actores, salario_diario=1):
    """
    Calcula el coste total de una configuración de sesiones de doblaje.
    """
    costes_por_sesion = []
    
    for sesion in configuracion:
        actores_sesion = set()
        for toma in sesion:
            actores_sesion.update(tomas_actores[toma])
        costes_por_sesion.append(len(actores_sesion) * salario_diario)
    
    coste_total = sum(costes_por_sesion)
    return (coste_total, costes_por_sesion)

def resumen_configuracion(configuracion, tomas_actores):
    """
    Genera un resumen detallado de la configuración.
    """
    coste_total, costes_por_sesion = coste(configuracion, tomas_actores)
    
    print("RESUMEN DE CONFIGURACIÓN")
    print("="*30)
    
    for i, (sesion, coste_dia) in enumerate(zip(configuracion, costes_por_sesion), 1):
        actores_sesion = set()
        for toma in sesion:
            actores_sesion.update(tomas_actores[toma])
        
        print(f"Sesión {i}: tomas {sesion}")
        print(f"         actores {sorted(actores_sesion)} → {coste_dia} días-actor")
    
    print("="*30)
    print(f"TOTAL: {coste_total} días-actor")
    
    return coste_total

# EJEMPLO
config_secuencial = [
    [1, 2, 3, 4, 5, 6],       # Día 1
    [7, 8, 9, 10, 11, 12],    # Día 2  
    [13, 14, 15, 16, 17, 18], # Día 3
    [19, 20, 21, 22, 23, 24], # Día 4
    [25, 26, 27, 28, 29, 30]  # Día 5
]

coste_seq = resumen_configuracion(config_secuencial, tomas_actores)
print(f"Coste secuencial: {coste_seq} días-actor\n")

RESUMEN DE CONFIGURACIÓN
Sesión 1: tomas [1, 2, 3, 4, 5, 6]
         actores [1, 2, 3, 4, 5, 7, 8] → 7 días-actor
Sesión 2: tomas [7, 8, 9, 10, 11, 12]
         actores [1, 2, 3, 4, 5, 6, 8, 9] → 8 días-actor
Sesión 3: tomas [13, 14, 15, 16, 17, 18]
         actores [1, 2, 3, 4, 5, 6, 7, 10] → 8 días-actor
Sesión 4: tomas [19, 20, 21, 22, 23, 24]
         actores [1, 2, 3, 4, 5, 6, 8] → 7 días-actor
Sesión 5: tomas [25, 26, 27, 28, 29, 30]
         actores [1, 2, 3, 4, 5, 6, 9, 10] → 8 días-actor
TOTAL: 38 días-actor
Coste secuencial: 38 días-actor



Esta función objetivo proporciona la métrica necesaria para evaluar cualquier configuración propuesta y será fundamental para implementar algoritmos que comparen las posibles configuraciones buscando minimizar el coste total.

## ¿Es un problema de maximización o minimización?



Este caso es un problema de minimización. El objetivo consiste en encontrar la configuración de sesiones de doblaje que minimice el coste total expresado en días-actor.
El enunciado pide que se busque planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible, por tanto estamos ante minimización de costes.

---

### Diferenciación con problemas de maximización

Por el contrario, si fuera un problema de maximización, estariamos hablando de:
- Maximizar la utilización de actores por sesión.
- Maximizar el número de tomas por sesión.
- Maximizar la eficiencia temporal del proceso.


### Verificación 

La configuración secuencial da como resultado 38 días-actor. El objetivo del  algoritmo de optimización será encontrar configuraciones alternativas que produzcan un coste < 38 días-actor, confirmando que es un problema de minimización.



## Diseña un algoritmo para resolver el problema por fuerza bruta



El algoritmo de fuerza bruta evalúa todas las configuraciones posibles y selecciona la de menor coste.

### Algoritmo conceptual
Para poder explorar todo el espacio de soluciones reducimos el problema original (30 tomas, ≤ 6 por sesión) a uno manejable (8 tomas, ≤ 3 por sesión):

1. Generar todas las formas de agrupar las 8 tomas en sesiones de ≤ 3 tomas (particiones).
2. Calcular para cada configuración su coste en días-actor con la función coste.
3. Seleccionar la configuración con menor coste (óptimo global).

### Implementación (problema reducido)
- Existen 521 640 configuraciones posibles.
- Las particiones se generan recursivamente con itertools.combinations; el coste se calcula mediante set().union(*).
- El óptimo obtenido sirve de referencia para medir la brecha (gap) de cualquier heurística (por ejemplo, el algoritmo greedy).

In [103]:
# Datos del problema reducido
TOMAS_ACTORES_SIMPLIFICADO = {
    1: {1, 2}, 
    2: {2, 3}, 
    3: {1, 3, 4}, 
    4: {1, 2, 4},
    5: {2, 3}, 
    6: {1, 4}, 
    7: {3, 4}, 
    8: {1, 2, 3}
}

def fuerza_bruta(datos_tomas=None):
    if datos_tomas is None:
        datos_tomas = tomas_actores
    
    tomas = list(datos_tomas.keys())
    max_por_sesion = 3 if len(tomas) <= 10 else 6
    
    mejor_conf, mejor_coste = None, inf

    # Pila explícita para generar todas las particiones
    pila = [([], tomas)]

    while pila:
        config, restantes = pila.pop()

        # Si ya no quedan tomas, evaluamos el coste de la configuración completa
        if not restantes:
            coste = sum(
                len(set().union(*[datos_tomas[toma] for toma in sesion]))
                for sesion in config
            )
            if coste < mejor_coste:
                mejor_conf, mejor_coste = config, coste
            continue

        # Generar los posibles bloques de la siguiente sesión
        for i in range(1, min(len(restantes), max_por_sesion) + 1):
            for ses in combinations(restantes, i):
                nuevos_restantes = [t for t in restantes if t not in ses]
                pila.append((config + [list(ses)], nuevos_restantes))

    return mejor_conf, mejor_coste

opt, c_opt = fuerza_bruta(TOMAS_ACTORES_SIMPLIFICADO)
print(f"Óptimo {c_opt}: {opt}")


Óptimo 9: [[3, 7, 8], [1, 4, 6], [2, 5]]


#### Algoritmo Teórico  (No ejecutable)

In [104]:
def fuerza_bruta_completo(tomas_actores):
    """
    Teóricamente evaluaría las 11_423_951_396_577_720 configuraciones
    (30 tomas en bloques ≤ 6).  **Inviable en la práctica.**
    """
    mejor_coste = float('inf')
    mejor_configuracion = None
    
    # Generar TODAS las configuraciones posibles
    for configuracion in todas_las_configuraciones_posibles():
        coste_actual, _ = coste(configuracion, tomas_actores)
        
        if coste_actual < mejor_coste:
            mejor_coste = coste_actual
            mejor_configuracion = configuracion
    
    return mejor_configuracion, mejor_coste

El algoritmo es conceptualmente simple pero computacionalmente inviable, justificando la necesidad de algoritmos más eficientes.

## Calcula la complejidad del algoritmo por fuerza bruta


### Análisis de Complejidad del Algoritmo Implementado

El algoritmo de fuerza bruta se basa en probar todas las combinaciones posibles para poder encontrar la solución óptima, sin tener en cuenta la eficiencia del algoritmo. La complejidad del algoritmo depende principalmente de dos operaciones. 

**Componentes del algoritmo:**

1. Generación de las permutaciones.

Se generan todas las permutaciones posibles de las tomas. El número total de tomas es n = 30. Por tanto, el número total de permutaciones es n! = 30!.
No obstante, no se están generando todas las permutaciones posibles ya que se está limitando las soluciones ensayadas a num_perm. Por tanto:

- Si num_perm < n!, entonces la complejidad  es O(num_perm)
- Si ensayamos todas las permutaciones, la complejidad sería O(n!)

2. División y cálculo del coste para cada solución

Para cada solución generada, se divide las tomas en sesiones. Entonces, se calcula el coste asociado con esa permutación utilizando la función coste().
Esta operación tiene una complejidad que es función de n y k (es decir, el número máximo de tomas por sesión). Sin embargo, como n y k son constantes para cada iteración, la complejidad de esta operación es de orden O(1) para una solución dada.

### Complejidad Total del Algoritmo
Por tanto, multiplicando la complejidad de ambas operaciones, resulta que la complejidad de este algoritmo es:
- T(n) = O(num_perm × 1) = O(num_perm)

En el caso más general (evaluando todas las permutaciones):
- T(n) = O(n!)

Valores Numéricos Específicos
- Para n = 30 (30 tomas):

30! = 265,252,859,812,191,058,636,308,480,000,000 permutaciones
- Complejidad: O(2.65 × 10^32) en el peor caso

### Tiempo de Ejecución Estimado

Asumiendo 1 microsegundo por evaluación de permutación:


In [105]:
permutaciones_totales = math.factorial(30)  # 30!
evaluaciones_por_segundo = 1000000          # 1 microsegundo cada una

tiempo_segundos = permutaciones_totales / evaluaciones_por_segundo
tiempo_anos = tiempo_segundos / (365 * 24 * 3600)

print(tiempo_anos, "años")

8.411113007743248e+18 años


### Conclusión

El algoritmo de fuerza bruta implementado tiene:

- Complejidad temporal: O(num_perm) en general, O(n!) en el peor caso.
- Inviabilidad práctica: El tiempo de ejecución para evaluar todas las permutaciones sería extremadamente largo, por tanto seria inviable.

Esta complejidad justifica la necesidad de desarrollar algoritmos heurísticos que proporcionen soluciones de buena calidad en tiempo razonable, sacrificando la garantía de optimalidad por la viabilidad computacional.

## Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

### Primera opción: **Algoritmo Greedy**



Se ha diseñado un algoritmo greedy que resuelve el problema de coordinación de doblaje de manera eficiente, dando una alternativa práctica al algoritmo de fuerza bruta que es computacionalmente inviable.

#### Componentes del algoritmo

##### 1. **Construcción Greedy**
- **Selección inicial**: Comienza cada sesión con la toma que requiere menos actores.
- **Completado de sesión**: Añade tomas que minimicen el número de actores nuevos necesarios.
- **Heurística**: `actores_nuevos = actores_toma - actores_ya_presentes`.

##### 2. **Mejora Local**
- Intercambia tomas entre sesiones si reduce el coste total.
- Continúa hasta que no se encuentren más mejoras.
- Evita quedarse atrapado en la primera solución greedy.

##### 3. **Múltiples Ejecuciones**
- Ejecuta el proceso 3 veces .
- Retorna la mejor solución encontrada.

#### Argumentos de mejora respecto a fuerza bruta

##### **Viabilidad Computacional**

| Algoritmo | Complejidad | Tiempo estimado | Estado |
|-----------|-------------|-----------------|---------|
| **Fuerza Bruta** | O(n!) | Muchos años | **INVIABLE** |
| **Greedy** | O(n²) | <1 segundo | **VIABLE** |

La mejora principal es que transforma un problema inviable en uno práctico.


##### **Escalabilidad**

- **Fuerza bruta**: Crece exponencialmente → inviable para problemas mayores
- **Greedy**: Crece polinómicamente → escalable a producciones más grandes

#### Comparación Realista

- **Fuerza bruta**: Garantiza óptimo pero es imposible de ejecutar
- **Greedy**: Obtiene buenas soluciones en tiempo razonable

Como no podemos ejecutar el problema real por fuerza bruta real, comparamos contra:

1. **Configuración secuencial** (baseline de 38 días-actor)
2. **Muestreo aleatorio** (para demostrar que greedy > azar)
3. **Múltiples ejecuciones greedy** (para mostrar consistencia)

In [106]:
def coste(configuracion):
    """Calcula coste total: suma de actores únicos por sesión"""
    total = 0
    for sesion in configuracion:
        actores_sesion = set()
        for toma in sesion:
            actores_sesion.update(tomas_actores[toma])
        total += len(actores_sesion)
    return total

def algoritmo_greedy_mejorado():
    """Algoritmo greedy con mejora local - SE ADAPTA AL TAMAÑO DE LOS DATOS"""
    
    # DETECTAR AUTOMÁTICAMENTE EL TAMAÑO DE LOS DATOS
    num_tomas = len(tomas_actores)
    max_tomas = max(tomas_actores.keys()) if tomas_actores else 0
    
    # ADAPTAR PARÁMETROS SEGÚN EL TAMAÑO
    if num_tomas <= 10:
        max_por_sesion = 3  # Para datos pequeños
    else:
        max_por_sesion = 6  # Para datos grandes
    
    def construir_solucion():
        # USAR TODAS LAS TOMAS DISPONIBLES
        tomas_restantes = set(tomas_actores.keys())  # ← ADAPTADO
        configuracion = []
        
        while tomas_restantes:
            sesion = []
            # Primera toma: la de menos actores
            primera = min(tomas_restantes, key=lambda t: len(tomas_actores[t]))
            sesion.append(primera)
            tomas_restantes.remove(primera)
            
            # Completar sesión: añadir tomas que menos actores nuevos aporten
            while len(sesion) < max_por_sesion and tomas_restantes:  # ← ADAPTADO
                actores_actuales = set()
                for t in sesion:
                    actores_actuales.update(tomas_actores[t])
                
                mejor = min(tomas_restantes, 
                          key=lambda t: len(tomas_actores[t] - actores_actuales))
                sesion.append(mejor)
                tomas_restantes.remove(mejor)
            
            configuracion.append(sesion)
        return configuracion
    
    def mejora_local(config):
        mejor_coste = coste(config)
        mejora = True
        
        while mejora:
            mejora = False
            for i in range(len(config)):
                for j in range(i + 1, len(config)):
                    for toma_i in config[i]:
                        for toma_j in config[j]:
                            # Intercambiar
                            config[i].remove(toma_i)
                            config[i].append(toma_j)
                            config[j].remove(toma_j)
                            config[j].append(toma_i)
                            
                            if coste(config) < mejor_coste:
                                mejor_coste = coste(config)
                                mejora = True
                                break
                            else:
                                # Revertir
                                config[i].remove(toma_j)
                                config[i].append(toma_i)
                                config[j].remove(toma_i)
                                config[j].append(toma_j)
                        if mejora: break
                    if mejora: break
                if mejora: break
        return config
    
    # Ejecutar 3 veces y quedarse con la mejor
    mejor_coste = float('inf')
    mejor_config = None
    
    for _ in range(3):
        config = construir_solucion()
        config = mejora_local(config)
        coste_actual = coste(config)
        
        if coste_actual < mejor_coste:
            mejor_coste = coste_actual
            mejor_config = config
    
    return mejor_config, mejor_coste

def muestreo_aleatorio(num_eval=1000):
    """Muestreo aleatorio para comparación - SE ADAPTA AL TAMAÑO DE LOS DATOS"""
    mejor_coste = float('inf')
    mejor_config = None
    
    # DETECTAR AUTOMÁTICAMENTE EL TAMAÑO
    tomas_disponibles = list(tomas_actores.keys())  # ← ADAPTADO
    num_tomas = len(tomas_disponibles)
    
    # ADAPTAR PARÁMETROS
    if num_tomas <= 10:
        max_por_sesion = 3
    else:
        max_por_sesion = 6
    
    for _ in range(num_eval):
        tomas = tomas_disponibles.copy()  # ← ADAPTADO
        random.shuffle(tomas)
        
        config = []
        for i in range(0, len(tomas), max_por_sesion):  # ← ADAPTADO
            sesion = tomas[i:i+max_por_sesion]
            if sesion:  # Solo añadir si no está vacía
                config.append(sesion)
        
        coste_actual = coste(config)
        if coste_actual < mejor_coste:
            mejor_coste = coste_actual
            mejor_config = config
    
    return mejor_config, mejor_coste

def comparar():
    """Función de comparación - SE ADAPTA AL TAMAÑO DE LOS DATOS"""
    
    # DETECTAR PARÁMETROS AUTOMÁTICAMENTE
    tomas_disponibles = list(tomas_actores.keys())
    num_tomas = len(tomas_disponibles)
    
    if num_tomas <= 10:
        max_por_sesion = 3
    else:
        max_por_sesion = 6
    
    print(f" DATOS DETECTADOS: {num_tomas} tomas, máx {max_por_sesion} por sesión")
    
    # Configuración secuencial ADAPTADA
    config_base = []
    for i in range(0, len(tomas_disponibles), max_por_sesion):  # ← ADAPTADO
        sesion = tomas_disponibles[i:i+max_por_sesion]
        if sesion:
            config_base.append(sesion)
    
    coste_base = coste(config_base)
    
    # Algoritmos (ahora adaptables)
    config_greedy, coste_greedy = algoritmo_greedy_mejorado()
    config_aleatorio, coste_aleatorio = muestreo_aleatorio()
    
    print("RESULTADOS:")
    print(f"Secuencial:     {coste_base} días-actor")
    print(f"Greedy:         {coste_greedy} días-actor")
    print(f"Aleatorio:      {coste_aleatorio} días-actor")
    
    mejora = ((coste_base - coste_greedy) / coste_base) * 100
    print('')
    print(f"Mejora Greedy: {mejora:.1f}%")
    
    print(f"\nConfiguración Greedy:")
    for i, sesion in enumerate(config_greedy, 1):
        actores = set()
        for toma in sesion:
            actores.update(tomas_actores[toma])
        print(f"   Día {i}: {sesion} → {len(actores)} actores")


# Con datos originales (30 tomas, máx 6 por sesión)
print("PRUEBA CON DATOS ORIGINALES:")
comparar()


PRUEBA CON DATOS ORIGINALES:
 DATOS DETECTADOS: 30 tomas, máx 6 por sesión
RESULTADOS:
Secuencial:     38 días-actor
Greedy:         28 días-actor
Aleatorio:      33 días-actor

Mejora Greedy: 26.3%

Configuración Greedy:
   Día 1: [28, 2, 17, 19, 13, 27] → 4 actores
   Día 2: [24, 12, 18, 14, 9, 23] → 5 actores
   Día 3: [30, 4, 5, 16, 25, 15] → 6 actores
   Día 4: [6, 1, 22, 7, 3, 20] → 6 actores
   Día 5: [10, 29, 26, 11, 21, 8] → 7 actores


### Resultados

- **Coste**: 28 días-actor (mejora del 26.3%)
- **Tiempo**: <1 segundo
- **Superioridad sobre azar**: Mejor que muestreo aleatorio de 1000 configuraciones

### Conclusión

El algoritmo greedy no solo mejora la complejidad teórica del fuerza bruta, sino que hace posible resolver el problema en la práctica. La verdadera mejora no es solo de eficiencia, sino de viabilidad computacional: convierte un problema imposible de resolver en uno que se puede abordar efectivamente.


### Segunda opción: **Algoritmo Genético Híbrido**

Se ha diseñado un algoritmo genético híbrido que resuelve el problema mediante evolución computacional adaptativa, proporcionando soluciones de alta calidad para el problema matemáticamente inviable  por fuerza bruta.

## Componentes del algoritmo

**1. Generación Diversificada Multi-Estrategia**
- 4 estrategias simultáneas: Greedy puro (25%), clustering por frecuencia de actores (25%), clustering por similitud (25%), y exploración aleatoria inteligente (25%).
- **Población adaptativa**: 100 individuos para problemas pequeños (≤10 tomas), 500 individuos para problemas grandes.
- Diversidad: Cada estrategia aporta perspectivas únicas de optimización.

**2. Evolución Adaptativa**
- **Generaciones variables**: 100 para problemas pequeños, 200 para problemas grandes.
- Selección por torneo competitivo: Torneos de 5 individuos para máxima presión selectiva.
- Elitismo 20%: Preserva las mejores soluciones de cada generación.

**3. Cruzamiento Inteligente Adaptativo**
- **Preservación de sesiones óptimas** (60%): Combina las mejores sesiones de ambos padres sin duplicados.
- **Intercambio de tomas** (40%): Transferencia directa de tomas individuales entre configuraciones.
- **Reparación automática**: Garantiza configuraciones válidas sin tomas duplicadas mediante validación y reparación exhaustiva.

**4. Mutación Dual**
- **Intercambios múltiples** (50%): 1-4 intercambios aleatorios entre sesiones.
- **Búsqueda local 2-opt** (50%): Evaluación de intercambios vecinos.
- Tasa de mutación: 50% para mantener diversidad.

**5. Búsqueda Local Exhaustiva**
- **2-opt completo**: Prueba TODOS los intercambios posibles entre tomas.
- **50 pasadas iterativas**: Continúa hasta convergencia total sin mejoras.
- **Garantía de óptimo local**: Asegura que no existan mejoras simples disponibles.

**6. Multi-Restart Adaptativo**
- **Ejecuciones variables**: 1 restart para problemas pequeños, 3 para problemas grandes.
- **Búsqueda híbrida**: Genético + búsqueda local exhaustiva por restart.
- **Selección del mejor global**: Conserva la mejor solución de todas las ejecuciones.

## Argumentos de mejora respecto a fuerza bruta

**Escalabilidad**
- Fuerza bruta: O(e^n) → imposible para problemas reales.
- Algoritmo genético híbrido: O(n²) → escalable y práctico.

**Calidad**
- Fuerza bruta: Garantiza óptimo global teórico pero es físicamente imposible de ejecutar.
- Algoritmo genético híbrido: **Encuentra el óptimo global** en problemas pequeños verificables, y soluciones de alta calidad en problemas grandes.

**Adaptabilidad**
- **Detección automática** del tamaño del problema.
- **Ajuste dinámico** de parámetros según la complejidad.
- **Validación exhaustiva** para garantizar configuraciones completas.

**Comparación**

Como la fuerza bruta es computacionalmente imposible para problemas grandes, comparamos contra:
- Configuración secuencial (baseline adaptativo).
- Algoritmos heurísticos (greedy, aleatorio) para demostrar superioridad.
- **Fuerza bruta en problemas pequeños** (para verificación de optimalidad).
- Múltiples restarts independientes (para mostrar robustez y consistencia).

In [107]:
def algoritmo_genetico_super_intensivo():    
    # Detectar parámetros
    tomas_list = list(tomas_actores.keys())
    num_tomas = len(tomas_list)
    max_por_sesion = 3 if num_tomas <= 10 else 6
    
    if num_tomas <= 10:
        poblacion, generaciones = 100, 100
    else:
        poblacion, generaciones = 500, 200
        
    # Función de coste
    def coste(config):
        # VERIFICAR QUE TODAS LAS TOMAS ESTÉN INCLUIDAS
        tomas_en_config = set([t for sesion in config for t in sesion])
        if len(tomas_en_config) != num_tomas:
            return float('inf')  # PENALIZAR CONFIGURACIONES INCOMPLETAS
        
        return sum(len(set().union(*[tomas_actores[t] for t in s])) for s in config)
    
    # VAalidar configuiración completa
    def validar_y_reparar(config):
        """Asegura que la configuración incluya TODAS las tomas"""
        tomas_usadas = set([t for sesion in config for t in sesion])
        faltantes = set(tomas_list) - tomas_usadas
        
        if faltantes:
            # Reparar añadiendo tomas faltantes
            faltantes_list = list(faltantes)
            random.shuffle(faltantes_list)
            
            # Intentar añadir a sesiones existentes
            for toma in faltantes_list[:]:
                for sesion in config:
                    if len(sesion) < max_por_sesion:
                        sesion.append(toma)
                        faltantes_list.remove(toma)
                        break
            
            # Si quedan faltantes, crear nuevas sesiones
            while faltantes_list:
                nueva_sesion = faltantes_list[:max_por_sesion]
                config.append(nueva_sesion)
                faltantes_list = faltantes_list[max_por_sesion:]
        
        # Eliminar duplicados
        tomas_vistas = set()
        for sesion in config:
            sesion_sin_duplicados = []
            for toma in sesion:
                if toma not in tomas_vistas:
                    sesion_sin_duplicados.append(toma)
                    tomas_vistas.add(toma)
            sesion[:] = sesion_sin_duplicados
        
        # Eliminar sesiones vacías
        config[:] = [s for s in config if s]
        
        return config
    
    # Generación
    def generar():
        r = random.random()
        if r < 0.25:  # Greedy puro
            restantes, config = set(tomas_list), []
            while restantes:
                s = [min(restantes, key=lambda t: len(tomas_actores[t]))]
                restantes.remove(s[0])
                while len(s) < max_por_sesion and restantes:
                    a = set().union(*[tomas_actores[t] for t in s])
                    m = min(restantes, key=lambda t: len(tomas_actores[t] - a))
                    s.append(m)
                    restantes.remove(m)
                config.append(s)
            return validar_y_reparar(config)
            
        elif r < 0.5:  # Por frecuencia de actores
            f = {}
            for actores in tomas_actores.values():
                for actor in actores:
                    f[actor] = f.get(actor, 0) + 1
            
            config, usado = [], set()
            actores_freq = sorted(f.keys(), key=f.get, reverse=True)
            
            # Crear sesiones por actor frecuente
            for actor in actores_freq:
                if len(usado) >= num_tomas:  # Ya tenemos todas las tomas
                    break
                    
                tomas_actor = [t for t in tomas_list if actor in tomas_actores[t] and t not in usado]
                if tomas_actor:
                    sesion = tomas_actor[:max_por_sesion]
                    config.append(sesion)
                    usado.update(sesion)
            
            # Asegurar tomas
            restantes = [t for t in tomas_list if t not in usado]
            if restantes:
                random.shuffle(restantes)
                for i in range(0, len(restantes), max_por_sesion):
                    config.append(restantes[i:i+max_por_sesion])
            
            return validar_y_reparar(config)
            
        elif r < 0.75:  # Clustering por similitud
            config, restantes = [], set(tomas_list)
            
            while restantes:
                s = [random.choice(list(restantes))]
                restantes.remove(s[0])
                
                while len(s) < max_por_sesion and restantes:
                    sim = {}
                    for t in restantes:
                        sim[t] = sum(len(tomas_actores[t] & tomas_actores[z]) for z in s)
                    if sim:
                        m = max(sim, key=sim.get)
                        s.append(m)
                        restantes.remove(m)
                    else:
                        break
                config.append(s)
            
            return validar_y_reparar(config)
            
        else:  # Aleatorio
            t = tomas_list.copy()
            random.shuffle(t)
            config = [t[i:i+max_por_sesion] for i in range(0, len(t), max_por_sesion)]
            return validar_y_reparar(config)
    
    # Cruzamiento
    def cruzar(p1, p2):
        if random.random() < 0.6:
            # Cruzamiento por mejores sesiones
            todas = [(s, -len(set().union(*[tomas_actores[t] for t in s]))) for s in p1 + p2]
            todas.sort(key=lambda x: x[1], reverse=True)
            hijo, usado = [], set()
            
            for s, _ in todas:
                if not any(t in usado for t in s):
                    hijo.append(s[:])
                    usado.update(s)
                    if len(usado) >= num_tomas:
                        break
            
            return validar_y_reparar(hijo)
        else:
            # Cruzamiento por intercambio
            hijo = deepcopy(p1)
            for _ in range(random.randint(2, 6)):
                if len(hijo) >= 2 and len(p2) >= 2:
                    s1 = random.randint(0, len(hijo)-1)
                    s2 = random.randint(0, len(p2)-1)
                    if hijo[s1] and p2[s2]:
                        pos1 = random.randint(0, len(hijo[s1])-1)
                        pos2 = random.randint(0, len(p2[s2])-1)
                        hijo[s1][pos1] = p2[s2][pos2]
            
            return validar_y_reparar(hijo)
    
    # Mutaciión
    def mutar(ind):
        if random.random() < 0.5:
            return ind
        
        config = deepcopy(ind)
        r = random.random()
        
        if r < 0.5:  # Intercambios múltiples
            for _ in range(random.randint(1, 4)):
                if len(config) >= 2:
                    s1, s2 = random.sample(range(len(config)), 2)
                    if config[s1] and config[s2]:
                        p1 = random.randint(0, len(config[s1])-1)
                        p2 = random.randint(0, len(config[s2])-1)
                        config[s1][p1], config[s2][p2] = config[s2][p2], config[s1][p1]
        else:  # 2-opt local
            if len(config) >= 2:
                i, j = random.sample(range(len(config)), 2)
                if config[i] and config[j]:
                    ti = random.randint(0, len(config[i])-1)
                    tj = random.randint(0, len(config[j])-1)
                    config[i][ti], config[j][tj] = config[j][tj], config[i][ti]
        
        return validar_y_reparar(config)
    
    # Evolución
    pob = [generar() for _ in range(poblacion)]
    mejor, mejor_c = None, float('inf')
    
    for gen in range(generaciones):
        fit = [coste(ind) for ind in pob]
        mi = fit.index(min(fit))
        if fit[mi] < mejor_c:
            mejor_c, mejor = fit[mi], deepcopy(pob[mi])
        
        
        # Nueva generación
        elite = int(len(pob) * 0.2)
        ei = sorted(range(len(fit)), key=lambda i: fit[i])[:elite]
        nueva = [deepcopy(pob[i]) for i in ei]
        
        while len(nueva) < len(pob):
            t1 = random.sample(list(zip(pob, fit)), 5)
            t2 = random.sample(list(zip(pob, fit)), 5)
            p1, p2 = min(t1, key=lambda x: x[1])[0], min(t2, key=lambda x: x[1])[0]
            nueva.append(mutar(cruzar(p1, p2)))
        
        pob = nueva
    
    return mejor, mejor_c

def busqueda_local_exhaustiva(config):
    """Búsqueda local 2-opt EXHAUSTIVA"""
    mejor, mc, mej = deepcopy(config), 0, 0
    
    # Verificar configuración
    tomas_en_config = set([t for sesion in mejor for t in sesion])
    if len(tomas_en_config) != len(tomas_actores):
        print(f" Configuración inicial incompleta: {len(tomas_en_config)}/{len(tomas_actores)} tomas")
        return mejor, float('inf')
    
    mc = sum(len(set().union(*[tomas_actores[t] for t in s])) for s in mejor)
    
    for _ in range(50):
        cambio = False
        for i in range(len(mejor)):
            for j in range(i+1, len(mejor)):
                for ti in range(len(mejor[i])):
                    for tj in range(len(mejor[j])):
                        # Intercambiar
                        mejor[i][ti], mejor[j][tj] = mejor[j][tj], mejor[i][ti]
                        nc = sum(len(set().union(*[tomas_actores[t] for t in s])) for s in mejor)
                        if nc < mc:
                            mc, mej, cambio = nc, mej+1, True
                        else:
                            # Revertir
                            mejor[i][ti], mejor[j][tj] = mejor[j][tj], mejor[i][ti]
        if not cambio:
            break
    
    return mejor, mc

def ejecutar_super_intensivo():
    """Ejecuta el algoritmo"""
    
    # Baseline
    tomas_list = list(tomas_actores.keys())
    max_por_sesion = 3 if len(tomas_list) <= 10 else 6
    baseline = [tomas_list[i:i+max_por_sesion] for i in range(0, len(tomas_list), max_por_sesion)]
    coste_baseline = sum(len(set().union(*[tomas_actores[t] for t in s])) for s in baseline)
    
    
    # Restarts
    num_restarts = 1 if len(tomas_list) <= 10 else 3
    random.seed(42)
    inicio = time.time()
    mejor_global, mejor_coste = None, float('inf')
    
    for restart in range(num_restarts):
        if num_restarts > 1:
            print(f"\n--- RESTART {restart+1}/{num_restarts} ---")
        
        config_gen, coste_gen = algoritmo_genetico_super_intensivo()
        config_final, coste_final = busqueda_local_exhaustiva(config_gen)
        
        print(f"Resultado: {coste_final:.3f} días-actor")
        
        if coste_final < mejor_coste:
            mejor_coste, mejor_global = coste_final, config_final
            if num_restarts > 1:
                print("Mejora")
    
    tiempo = time.time() - inicio
    
    # Resuiltados
    print(f"\nRESULTADO FINAL: {mejor_coste:.3f} días-actor")
    for i, s in enumerate(mejor_global, 1):
        a = set().union(*[tomas_actores[t] for t in s])
        print(f"  Día {i}: {s} → {len(a)} actores")
    
    mejora = ((coste_baseline - mejor_coste) / coste_baseline) * 100
    print(f"\nBaseline: {coste_baseline} → Optimizado: {mejor_coste:.3f}")
    print(f"Mejora: {mejora:.1f}% en {tiempo:.1f}s")
        
    
    return mejor_global, mejor_coste

ejecutar_super_intensivo()


--- RESTART 1/3 ---
Resultado: 27.000 días-actor
Mejora

--- RESTART 2/3 ---
Resultado: 27.000 días-actor

--- RESTART 3/3 ---
Resultado: 28.000 días-actor

RESULTADO FINAL: 27.000 días-actor
  Día 1: [18, 14, 24, 17, 19, 23] → 3 actores
  Día 2: [16, 25, 6, 30, 9, 13] → 5 actores
  Día 3: [5, 2, 22, 1, 11, 28] → 6 actores
  Día 4: [8, 3, 4, 15, 29, 21] → 6 actores
  Día 5: [26, 20, 10, 12, 27, 7] → 7 actores

Baseline: 38 → Optimizado: 27.000
Mejora: 28.9% en 14.5s


([[18, 14, 24, 17, 19, 23],
  [16, 25, 6, 30, 9, 13],
  [5, 2, 22, 1, 11, 28],
  [8, 3, 4, 15, 29, 21],
  [26, 20, 10, 12, 27, 7]],
 27)

### Comparaciones datos simples

In [108]:
# Datos de prueba
TOMAS_ACTORES_SIMPLIFICADO = {
  1: {1, 2}, 
  2: {2, 3}, 
  3: {1, 3, 4}, 
  4: {1, 2, 4},
  5: {2, 3}, 
  6: {1, 4}, 
  7: {3, 4}, 
  8: {1, 2, 3}
}

# Cambiar temporalmente los datos
tomas_actores_backup = tomas_actores.copy()
tomas_actores = TOMAS_ACTORES_SIMPLIFICADO

# Ejecutar secuencial (baseline)
tomas_list = list(tomas_actores.keys())
max_por_sesion = 3 if len(tomas_list) <= 10 else 6
config_secuencial = [tomas_list[i:i+max_por_sesion] for i in range(0, len(tomas_list), max_por_sesion)]
coste_secuencial = sum(len(set().union(*[tomas_actores[t] for t in s])) for s in config_secuencial)
print(f"\n- Secuencial: {coste_secuencial} días-actor")
print(f"     - Configuración: {config_secuencial}")

# Ejecutar fuerza bruta 
config_fb, coste_fb = fuerza_bruta()
print(f"\n- Fuerza bruta: {coste_fb} días-actor")
print(f"     - Configuración: {config_fb}")

# Ejecutar greedy
config_greedy, coste_greedy = algoritmo_greedy_mejorado()
print(f"\n- Greedy: {coste_greedy} días-actor")
print(f"     - Configuración: {config_greedy}")

# Ejecutar aleatorio
random.seed(42)
tomas_aleatorio = tomas_list.copy()
random.shuffle(tomas_aleatorio)
config_aleatorio = [tomas_aleatorio[i:i+max_por_sesion] for i in range(0, len(tomas_aleatorio), max_por_sesion)]
coste_aleatorio = sum(len(set().union(*[tomas_actores[t] for t in s])) for s in config_aleatorio)
print(f"\n- Aleatorio: {coste_aleatorio} días-actor")
print(f"     - Configuración: {config_aleatorio}")

# Ejecutar algoritmo genético (CON LOS MISMOS DATOS)
random.seed(42)
config_genetico, coste_genetico = algoritmo_genetico_super_intensivo()
config_genetico_final, coste_genetico_final = busqueda_local_exhaustiva(config_genetico)
print(f"\n- Genético: {coste_genetico_final} días-actor")
print(f"     - Configuración: {config_genetico_final}")

# Tabla comparativa
print(f"\n RESUMEN COMPARATIVO:")
print(f"{'Algoritmo':<12} {'Coste':<6} ")
print("-" * 25)
print(f"{'Secuencial':<12} {coste_secuencial:<6} ")
print(f"{'Fuerza Bruta':<12} {coste_fb:<6} ")
print(f"{'Greedy':<12} {coste_greedy:<6} ")
print(f"{'Aleatorio':<12} {coste_aleatorio:<6} ")
print(f"{'Genético':<12} {coste_genetico_final:<6} ")

# Restaurar datos originales AL FINAL
tomas_actores = tomas_actores_backup


- Secuencial: 12 días-actor
     - Configuración: [[1, 2, 3], [4, 5, 6], [7, 8]]

- Fuerza bruta: 9 días-actor
     - Configuración: [[3, 7, 8], [1, 4, 6], [2, 5]]

- Greedy: 10 días-actor
     - Configuración: [[1, 2, 5], [6, 3, 7], [4, 8]]

- Aleatorio: 11 días-actor
     - Configuración: [[4, 5, 7], [8, 3, 6], [1, 2]]

- Genético: 9 días-actor
     - Configuración: [[4, 1, 8], [7, 3, 6], [5, 2]]

 RESUMEN COMPARATIVO:
Algoritmo    Coste  
-------------------------
Secuencial   12     
Fuerza Bruta 9      
Greedy       10     
Aleatorio    11     
Genético     9      


In [109]:
# Cambiar temporalmente los datos
tomas_actores_backup = tomas_actores.copy()
# tomas_actores = tomas_actores  # Esta línea no hace nada, la elimino

# Ejecutar secuencial (baseline)
tomas_list = list(tomas_actores.keys())
max_por_sesion = 3 if len(tomas_list) <= 10 else 6
config_secuencial = [tomas_list[i:i+max_por_sesion] for i in range(0, len(tomas_list), max_por_sesion)]
coste_secuencial = sum(len(set().union(*[tomas_actores[t] for t in s])) for s in config_secuencial)
print(f"\n- Secuencial: {coste_secuencial} días-actor")
print(f"     - Configuración: {config_secuencial}")

# Ejecutar greedy (CON LOS MISMOS DATOS)
config_greedy, coste_greedy = algoritmo_greedy_mejorado()
print(f"\n- Greedy: {coste_greedy} días-actor")
print(f"     - Configuración: {config_greedy}")

# Ejecutar aleatorio
random.seed(42)
tomas_aleatorio = tomas_list.copy()
random.shuffle(tomas_aleatorio)
config_aleatorio = [tomas_aleatorio[i:i+max_por_sesion] for i in range(0, len(tomas_aleatorio), max_por_sesion)]
coste_aleatorio = sum(len(set().union(*[tomas_actores[t] for t in s])) for s in config_aleatorio)
print(f"\n- Aleatorio: {coste_aleatorio} días-actor")
print(f"     - Configuración: {config_aleatorio}")

# Ejecutar algoritmo genético (CON LOS MISMOS DATOS)
random.seed(42)
config_genetico, coste_genetico = algoritmo_genetico_super_intensivo()
config_genetico_final, coste_genetico_final = busqueda_local_exhaustiva(config_genetico)
print(f"\n- Genético: {coste_genetico_final} días-actor")
print(f"     - Configuración: {config_genetico_final}")

# Tabla comparativa
print(f"\nRESUMEN COMPARATIVO:")
print(f"{'Algoritmo':<12} {'Coste':<6}")
print("-" * 25)
print(f"{'Secuencial':<12} {coste_secuencial:<6}")
print(f"{'Greedy':<12} {coste_greedy:<6} ")
print(f"{'Aleatorio':<12} {coste_aleatorio:<6} ")
print(f"{'Genético':<12} {coste_genetico_final:<6} ")

# Restaurar datos originales AL FINAL
tomas_actores = tomas_actores_backup


- Secuencial: 38 días-actor
     - Configuración: [[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12], [13, 14, 15, 16, 17, 18], [19, 20, 21, 22, 23, 24], [25, 26, 27, 28, 29, 30]]

- Greedy: 28 días-actor
     - Configuración: [[28, 2, 17, 19, 13, 27], [24, 12, 18, 14, 9, 23], [30, 4, 5, 16, 25, 15], [6, 1, 22, 7, 3, 20], [10, 29, 26, 11, 21, 8]]

- Aleatorio: 38 días-actor
     - Configuración: [[20, 15, 11, 27, 23, 7], [6, 13, 12, 16, 10, 26], [30, 22, 17, 28, 2, 14], [19, 3, 18, 29, 5, 25], [8, 9, 24, 1, 4, 21]]

- Genético: 27 días-actor
     - Configuración: [[18, 14, 24, 17, 19, 23], [16, 25, 6, 30, 9, 13], [5, 2, 22, 1, 11, 28], [8, 3, 4, 15, 29, 21], [26, 20, 10, 12, 27, 7]]

RESUMEN COMPARATIVO:
Algoritmo    Coste 
-------------------------
Secuencial   38    
Greedy       28     
Aleatorio    38     
Genético     27     


## Conclusiones

### Resultados Obtenidos

**Problema Pequeño (8 tomas):**
- **Genético**: 9 días-actor (óptimo global verificado)
- **Greedy**: 10 días-actor (+11.1% vs óptimo)
- **Fuerza Bruta**: 9 días-actor (referencia óptima)

**Problema Real (30 tomas):**
- **Genético**: 27 días-actor (mejor resultado)
- **Greedy**: 28 días-actor (+3.7% vs genético)
- **Baseline**: 38 días-actor (+40.7% vs genético)


El algoritmo genético encuentra el óptimo global en muchas ocasiones y supera consistentemente a las alternativas.



### Conclusión

**El algoritmo genético híbrido representa la solución óptima** para la coordinación de doblaje, combinando:
- ✅ **Calidad**: Encuentra óptimos o cuasi-óptimos
- ✅ **Eficiencia**: O(n²) vs O(e^n) de fuerza bruta
- ✅ **Adaptabilidad**: Funciona con cualquier configuración de datos

Respecto a nuestros resultados, la evolución inteligente + búsqueda local exhaustiva proporciona el mejor equilibrio entre calidad y velocidad para este tipo de problemas de optimización.

## Calcula la complejidad del algoritmo

### Greedy

### Análisis de Complejidad del Algoritmo Greedy 

#### **Complejidad Temporal**

El algoritmo tiene tres componentes principales:

##### **1. Construcción Greedy**
```python
while tomas_restantes:                  # 5 sesiones máximo
    primera = min(tomas_restantes)      # O(n)
    while len(sesion) < 6:              # 6 tomas máximo
        mejor = min(tomas_restantes)    # O(n)
```
**Complejidad**: O(5 × 6 × n) = **O(n)**

##### **2. Mejora Local**
```python
for i in range(sesiones):           # 5 sesiones
    for j in range(sesiones):       # 5 sesiones
        for toma_i in sesion:       # 6 tomas
            for toma_j in sesion:   # 6 tomas
                coste(config)       # O(n) - recalcula coste completo
```
**Complejidad por iteración**: O(5 × 5 × 6 × 6 × n) = **O(n)**

Pero se repite hasta convergencia (peor caso: n iteraciones)<br>
**Complejidad final**: O(n × n) = **O(n²)**


##### **3. Múltiples Ejecuciones**
```python
for _ in range(3):  # 3 intentos
    # construcción + mejora local
```
**Complejidad**: O(3 × n2) = **O(n²)**

#### **Complejidad Total**

| Fase | Complejidad |
|------|-------------|
| Construcción | O(n) |
| Mejora Local | O(n²) |
| **Total** | **O(n²)** |

#### **Complejidad Espacial**

```python
tomas_actores = {...}           # O(n) - datos entrada
configuracion = [[...], ...]    # O(n) - solución
variables_auxiliares            # O(1) - constante
```
**Complejidad espacial**: **O(n)**


### Conclusión

- **Complejidad temporal**: O(n²)
- **Complejidad espacial**: O(n)

### Hibrido genético

### Análisis de Complejidad del Algoritmo Genético Híbrido

#### **Complejidad Temporal**

##### **1. Generación de Población Inicial**
```python
# Población adaptativa (no fija de 500)
if num_tomas <= 10:
    poblacion = 100
else:
    poblacion = 500

# 4 estrategias con diferentes complejidades
def generar():
    # Greedy: O(n²) por construcción
    # Frecuencia: O(n) por conteo + O(n) por agrupación = O(n)  
    # Clustering: O(n²) por similitudes
    # Aleatorio: O(n)
```

**Complejidad**: O(poblacion × n²) = **O(n²)** (adaptativo)

##### **2. Bucle Evolutivo Principal**

```python
# Generaciones adaptativas (no fijas de 1000)
if num_tomas <= 10:
    generaciones = 100
else:
    generaciones = 200

for gen in range(generaciones):
    fit = [coste(ind) for ind in pob]   # poblacion × O(n)
```

**Complejidad por generación**: O(poblacion × n) = **O(n)**

##### **3. Operadores Genéticos por Generación**

```python
# Cruzamiento + Mutación
while len(nueva) < poblacion:
    cruzar(p1, p2)                      # O(n) - validación y reparación
    mutar(hijo)                         # O(n) - intercambios simples (no 2-opt complejo)
```

**Complejidad por generación**: O(poblacion × n) = **O(n)**

##### **4. Algoritmo Genético Completo**

```python
for gen in range(generaciones):         # 100-200 generaciones (adaptativo)
    # Evaluación: O(n)
    # Operadores: O(n)
```

**Complejidad**: O(generaciones × n) = **O(n)**

##### **5. Búsqueda Local Exhaustiva**

```python
for _ in range(50):                     # 50 pasadas máximo
    for i in range(sesiones):           # ~5 sesiones
        for j in range(i+1, sesiones):
            for ti in range(len(sesion)):  # ~6 tomas por sesión
                for tj in range(len(sesion)):
                    coste(config)       # O(n) - recalcula coste
```

**Complejidad**: O(50 × sesiones² × max_por_sesion² × n) = **O(n)**

##### **6. Multi-Restart Adaptativo**

```python
# Restarts adaptativos (no fijos de 3)
if num_tomas <= 10:
    num_restarts = 1
else:
    num_restarts = 3

for restart in range(num_restarts):
    algoritmo_genetico()                # O(n)
    busqueda_local()                    # O(n)
```

**Complejidad**: O(num_restarts × n) = **O(n)**

#### **Complejidad Total Corregida**

| Fase | Operaciones | Complejidad |
|------|-------------|-------------|
| Generación inicial | 100-500 individuos | O(n²) |
| Evolución genética | 100-200 gen × población | O(n) |
| Búsqueda local | 50 pasadas × 2-opt | O(n) |
| Multi-restart | 1-3 ejecuciones | O(n) |
| **Total** | **Adaptativo** | **O(n²)** |




#### **Conclusión**

* **Complejidad temporal**: **O(n²)** 
* **Complejidad espacial**: **O(n)** - 

El algoritmo mantiene complejidad polinómica con parámetros adaptativos, logrando el equilibrio entre calidad  y eficiencia computacional.

## Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Para validar el algoritmo greedy desarrollado, es necesario probar su comportamiento con diferentes configuraciones de datos. Para ello vamos a generar varios conjuntos de datos aleatorios que nos permitan evaluar el rendimiento del algoritmo.

### Parámetros del generador

El generador de datos aleatorios incorpora tiene parámetros para controlar las características del dataset:

- **Densidad de participación**: Probabilidad base de que un actor participe en una toma.
- **Distribución de tomas por actor**: Usando distribución de Poisson para simular patrones naturales.
- **Semilla de reproducibilidad**: Para garantizar resultados consistentes en las evaluaciones.

In [110]:
def generar_datos_aleatorios(densidad=0.3, seed=42):
    """Genera dataset aleatorio con la misma estructura que los datos originales"""
    random.seed(seed)
    datos_aleatorios = {}
    
    for toma in range(1, 31):
        # Número aleatorio de actores por toma (1-7 actores)
        num_actores = random.randint(1, min(7, int(densidad * 10) + 2))
        # Seleccionar actores aleatoriamente
        actores = set(random.sample(range(1, 11), num_actores))
        datos_aleatorios[toma] = actores
    
    return datos_aleatorios

# Generar tres datasets con diferentes características
datos_pocos_actores = generar_datos_aleatorios(densidad=0.2, seed=42)   # Pocas participaciones
datos_medios_actores = generar_datos_aleatorios(densidad=0.3, seed=123) # Participaciones medias  
datos_muchos_actores = generar_datos_aleatorios(densidad=0.5, seed=456) # Muchas participaciones

print("DATASETS GENERADOS:")

datasets = [
    ("Pocos actores", datos_pocos_actores),
    ("Medios actores", datos_medios_actores), 
    ("Muchos actores", datos_muchos_actores)
]

for nombre, datos in datasets:
    total_participaciones = sum(len(actores) for actores in datos.values())
    actores_por_toma = [len(actores) for actores in datos.values()]
    
    print(f"\n{nombre}:")
    print(f"  • Total participaciones: {total_participaciones}")
    print(f"  • Promedio actores/toma: {sum(actores_por_toma)/len(actores_por_toma):.1f}")
    print(f"  • Min-Max actores/toma: {min(actores_por_toma)}-{max(actores_por_toma)}")

DATASETS GENERADOS:

Pocos actores:
  • Total participaciones: 70
  • Promedio actores/toma: 2.3
  • Min-Max actores/toma: 1-4

Medios actores:
  • Total participaciones: 89
  • Promedio actores/toma: 3.0
  • Min-Max actores/toma: 1-5

Muchos actores:
  • Total participaciones: 123
  • Promedio actores/toma: 4.1
  • Min-Max actores/toma: 1-7


Esta generación de datos aleatorios permite evaluar la generalización del algoritmo y su comportamiento ante diferentes niveles de complejidad del problema.

## Aplica el algoritmo al juego de datos generado

Una vez generados los datasets aleatorios, vamos aevaluar el rendimiento del algoritmo greedy en cada uno de ellos.

In [111]:
def evaluar_dataset(datos_tomas_actores, nombre):
    """Evalúa el algoritmo greedy y genético en un dataset y compara resultados"""
    
    # Guardar datos originales
    global tomas_actores
    tomas_actores_backup = tomas_actores.copy()
    
    # Cambiar temporalmente a los nuevos datos
    tomas_actores = datos_tomas_actores
    
    try:
        # Configuración secuencial (baseline)
        tomas_ordenadas = list(range(1, 31))
        config_secuencial = [tomas_ordenadas[i:i+6] for i in range(0, 30, 6)]
        coste_secuencial = coste(config_secuencial)  # Usar función original
        
        # Usar algoritmo greedy original completo
        config_greedy, coste_greedy = algoritmo_greedy_mejorado()  # Función original
        
        # Ejecutar algoritmo genético
        random.seed(42)  # Para reproducibilidad
        config_genetico, coste_genetico = algoritmo_genetico_super_intensivo()
        config_genetico_final, coste_genetico_final = busqueda_local_exhaustiva(config_genetico)
        
        # Mejoras obtenidas
        mejora_greedy = ((coste_secuencial - coste_greedy) / coste_secuencial) * 100
        mejora_genetico = ((coste_secuencial - coste_genetico_final) / coste_secuencial) * 100
        
        return coste_secuencial, coste_greedy, coste_genetico_final, mejora_greedy, mejora_genetico, config_genetico_final
    
    finally:
        # Restaurar datos originales
        tomas_actores = tomas_actores_backup

# Evaluar datos originales
print("Evaluando datasets...")
resultado_original = evaluar_dataset(tomas_actores, "Datos Originales")

# Evaluar datasets aleatorios
datasets_aleatorios = [
    ("Pocos actores", datos_pocos_actores),
    ("Medios actores", datos_medios_actores),
    ("Muchos actores", datos_muchos_actores)
]

resultados = [resultado_original]
for nombre, datos in datasets_aleatorios:
    print(f"Procesando {nombre}...")
    resultado = evaluar_dataset(datos, nombre)
    resultados.append(resultado)

# Resumen
print(f"\n{'Dataset':<15} {'Secuencial':<11} {'Greedy':<8} {'Genético':<9} {'Mejora Greedy':<11} {'Mejora Genético':<12}")
print("-" * 75)

nombres = ["Originales", "Pocos", "Medios", "Muchos"]
for i, (sec, greedy, genetico, mejora_g, mejora_gen, config_gen) in enumerate(resultados):
    print(f"{nombres[i]:<15} {sec:>10d} {greedy:>7d} {genetico:>8d} {mejora_g:>9.1f}% {mejora_gen:>10.1f}%")



Evaluando datasets...
Procesando Pocos actores...
Procesando Medios actores...
Procesando Muchos actores...

Dataset         Secuencial  Greedy   Genético  Mejora Greedy Mejora Genético
---------------------------------------------------------------------------
Originales              38      28       27      26.3%       28.9%
Pocos                   40      29       26      27.5%       35.0%
Medios                  44      30       29      31.8%       34.1%
Muchos                  48      39       38      18.8%       20.8%


## Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo


https://www.geeksforgeeks.org/dsa/greedy-algorithms/

https://github.com/Jeffresh/Greedy-Algorithms

https://www.youtube.com/watch?v=TxEhI8ITKNA

https://www.youtube.com/watch?v=ENyox7kNKeY

https://www2.fiit.stuba.sk/~kvasnicka/Free%20books/Goldberg_Genetic_Algorithms_in_Search.pdf

**Referencias Académicas**

Cormen, T. H., Leiserson, C. E., Rivest, R. L., & Stein, C. (2009). Introduction to Algorithms (3rd ed.). MIT Press.

Johnson, D. S. (1973). "Near-optimal bin packing algorithms." Massachusetts Institute of Technology.


## Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Algunas de las opciones para continuar con el estudio del problema serían: 
- Comprobar cual es el rendimiento en otro tipo de algoritmos.

- Hacer el problema más realista con restricciones como: 

    - Actores no disponibles todos los días.
    - Actores con diferentes sueldos.  
    - Algunas tomas deben grabarse antes que otras.

- Probar con problemas más grandes y probar la escalabilidad, optimización real, etc

- Probar el algoritmo en problemas similares.
